In [67]:
from opg4py import graph_server
from pypgx import setloglevel
from getpass import getpass

setloglevel("ROOT", "WARN")

base_url = "http://localhost:7007"
username = "graphuser"
password = getpass("Password: ")

instance = graph_server.get_instance(base_url, username, password)
session = instance.create_session("test")
analyst = session.create_analyst()

print(session)

Password: ········
17:34:07.935 [main] WARN oracle.pgx.api.Pgx - connecting to a PGX server over unencrypted http; this configuration is insecure and must not be used in production
PgxSession(id: 527510d2-74d1-4361-98d4-a6a91e53bd3c, name: test)


In [68]:
graph = session.read_graph_by_name("BANK_GRAPH", "pg_sql")
print(graph)

PgxGraph(name: BANK_GRAPH, v: 360, e: 6230, directed: True, memory(Mb): 0)


In [58]:
graph.query_pgql("""
SELECT c.first_name, LABEL(e), a.acc_id
FROM MATCH (c:customer)-[e:owns]->(a:account)
WHERE c.cst_id = 10
""").print()

+--------------------------------+
| first_name | LABEL(e) | acc_id |
+--------------------------------+
| Laura      | OWNS     | 10     |
| Laura      | OWNS     | 90     |
+--------------------------------+



In [59]:
graph.query_pgql("""
SELECT ARRAY_AGG(a.acc_id) AS acc_ids
     , ARRAY_AGG(t.txn_id) AS txn_ids
     , ARRAY_AGG(t.amount) AS amounts
FROM MATCH TOP 1 SHORTEST (a1) (-[t:transferred_to]->(a))+ (a2)
WHERE a1.acc_id = 10 AND a2.acc_id = 50
  AND COUNT(a) = COUNT(DISTINCT a)
""").print()

+--------------------------------------+
| acc_ids  | txn_ids      | amounts    |
+--------------------------------------+
| [59, 50] | [1015, 5916] | [100, 500] |
+--------------------------------------+



In [60]:
graph.query_pgql("""
SELECT ARRAY_AGG(a.acc_id) AS acc_ids
     , ARRAY_AGG(t.txn_id) AS txn_ids
     , ARRAY_AGG(t.amount) AS amounts
FROM MATCH TOP 1 SHORTEST (a1) (-[t:transferred_to]->(a) WHERE t.amount > 800){5,} (a2)
WHERE a1.acc_id = 10 AND a2.acc_id = 50
  AND COUNT(a) = COUNT(DISTINCT a)
""").print()

+-------------------------------------------------------------------------------+
| acc_ids            | txn_ids                      | amounts                   |
+-------------------------------------------------------------------------------+
| [61, 9, 18, 5, 50] | [1047, 6114, 914, 1819, 512] | [900, 900, 900, 900, 900] |
+-------------------------------------------------------------------------------+



In [61]:
graph.query_pgql("""
SELECT ARRAY_AGG(a.acc_id) AS acc_ids
     , ARRAY_AGG(t.txn_id) AS txn_ids
     , ARRAY_AGG(t.amount) AS amounts
FROM MATCH TOP 1 SHORTEST (a1) (-[t:transferred_to]->(a) WHERE t.amount > 800){5,} (a1)
WHERE a1.acc_id = 10
  AND COUNT(a) = COUNT(DISTINCT a)
""").print()

+-----------------------------------------------------------------------------------+
| acc_ids              | txn_ids                        | amounts                   |
+-----------------------------------------------------------------------------------+
| [25, 20, 37, 77, 10] | [1028, 2514, 2005, 3712, 7714] | [900, 900, 900, 900, 900] |
+-----------------------------------------------------------------------------------+



In [62]:
graph.query_pgql("""
SELECT ELEMENT_NUMBER(e) AS element
     , src.acc_id AS src_acc_id
     , dst.acc_id AS dst_acc_id
     , e.amount
FROM MATCH TOP 1 SHORTEST (a1) (-[t:transferred_to]->(a) WHERE t.amount > 800){5,} (a1)
     ONE ROW PER STEP (src, e, dst)
WHERE a1.acc_id = 10
  AND COUNT(a) = COUNT(DISTINCT a)
ORDER BY element ASC
""").print()

+--------------------------------------------+
| element | src_acc_id | dst_acc_id | amount |
+--------------------------------------------+
| 2       | 10         | 25         | 900    |
| 4       | 25         | 20         | 900    |
| 6       | 20         | 37         | 900    |
| 8       | 37         | 77         | 900    |
| 10      | 77         | 10         | 900    |
+--------------------------------------------+



In [63]:
analyst.pagerank(graph)

VertexProperty(name: pagerank, type: double, graph: BANK_GRAPH)

In [64]:
analyst.degree_centrality(graph)
analyst.in_degree_centrality(graph)
analyst.out_degree_centrality(graph)

VertexProperty(name: out_degree, type: integer, graph: BANK_GRAPH)

In [65]:
graph.query_pgql("""
SELECT
  a.acc_id
, a.pagerank
, a.degree
, a.in_degree
, a.out_degree
FROM MATCH (a)
ORDER BY a.pagerank DESC
LIMIT 10
""").print()

+-----------------------------------------------------------------+
| acc_id | pagerank             | degree | in_degree | out_degree |
+-----------------------------------------------------------------+
| 41     | 0.009801832009337184 | 102    | 62        | 40         |
| 136    | 0.008582923494326107 | 94     | 44        | 50         |
| 123    | 0.00798299656203572  | 54     | 44        | 10         |
| 124    | 0.007960671512202215 | 72     | 42        | 30         |
| 132    | 0.007859597496691451 | 74     | 34        | 40         |
| 105    | 0.00758138767121552  | 88     | 38        | 50         |
| 33     | 0.007552475085421669 | 108    | 58        | 50         |
| 5      | 0.007491840249636888 | 92     | 42        | 50         |
| 24     | 0.007483037079652473 | 89     | 49        | 40         |
| 36     | 0.007448994936592109 | 71     | 41        | 30         |
+-----------------------------------------------------------------+

